# Metric Processing

This notebook collects and saved the attacking reults from multiple runs into a single file.


## Prerequisites

Conducted the Model Training and the Model Inversion metrics to produce reconstructed samples, as instructed in [README.md](https://github.com/ntuaislab/Trap-MID/blob/main/README.md). We used 5 runs for each experiments in our work.

In [ ]:
import json
import os

import pandas as pd

### Model Accuracy

#### Option 1

Create the json file to record the experiment and save the model accuracy with the following format:

```json
{
    "Model": [
        {
            "acc": 0.8783
        },
        {
            "acc": 0.8614
        },
        {
            "acc": 0.8587
        },
    ],
}
```

#### Option 2

Use the code in following cells to extract model accuracy from training logs.

In [ ]:
mode = 'BEST' # BEST, LAST
log_file = '<PATH_TO_TRAINING_LOG_FILE>'

with open(log_file, 'r') as f:
    lines = f.readlines()

if mode == 'BEST':
    line = lines[-1].strip() # Last line
    prefix = 'Best Acc:'
elif mode == 'LAST':
    line = lines[-2].strip() # Second last line
    prefix = 'Test Acc:'
else:
    raise NotImplementedError

acc = float(line[line.find(prefix)+len(prefix):].split('|')[0].strip())

In [ ]:
result_file = '<PATH_TO_RESULT_FILE>'
metric_name = 'Model'

if os.path.exists(result_file):
    with open(result_file, 'r') as f:
        results = json.load(f)
else:
    results = {}

if metric_name not in results:
    results[metric_name] = []
results[metric_name].append({ "acc": acc / 100  })

with open(result_file, 'w') as f:
    json.dump(results, f, indent=4)

### GMI, KED-MI, and PLG-MI

The following code shells shows the processing method of GMI, KED-MI, and PLG-MI results.

In [ ]:
experiment = '<EXPERIMENT_NAME>'

with open(f'<PATH_TO_ATTACK_RESULT>/{experiment}/result.json', 'r') as f:
    result = json.load(f)

In [ ]:
result_file = '<PATH_TO_RESULT_FILE>'
attack_name = 'GMI' # GMI, KED-MI, PLG-MI

if os.path.exists(result_file):
    with open(result_file, 'r') as f:
        results = json.load(f)
else:
    results = {}

if attack_name not in results:
    results[attack_name] = []
results[attack_name].append(result)

with open(result_file, 'w') as f:
    json.dump(results, f, indent=4)

### LOMMA

The following code shells shows the processing method of LONMMA results.

In [ ]:
experiment = '<EXPERIMENT_NAME>'
attack = 'gmi' # gmi, kedmi
model = 'VGG16'

df = pd.read_csv(f'<PATH_TO_LOMMA_RESULT>/{experiment}/{attack}_300ids/celeba_{model}/ours/Eval_results.csv')
result = {
    "acc": df['acc'][0] / 100,
    "acc5": df['acc5'][0] / 100,
    "var": (df['std'][0] / 100) ** 2,
    "var5": (df['std5'][0] / 100) ** 2,
    "knn_dist": df['knn'][0],
    "fid": df['fid'][0]
}

In [ ]:
result_file = '<PATH_TO_RESULT_FILE>'
attack_name = 'LOMMA (GMI)' # LOMMA (GMI), LOMMA (KED-MI)

if os.path.exists(result_file):
    with open(result_file, 'r') as f:
        results = json.load(f)
else:
    results = {}

if attack_name not in results:
    results[attack_name] = []
results[attack_name].append(result)

with open(result_file, 'w') as f:
    json.dump(results, f, indent=4)